In [1]:
# default_exp data.ts_dataset

In [1]:
#hide
%load_ext autoreload
%autoreload 2

In [2]:
#export
import numpy as np
import pandas as pd
import random
import torch as t

from fastcore.foundation import patch
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

In [3]:
#export
# TODO: rename x_s -> s_list
# TODO: rename ts_data -> ts_list
# TODO: resolver t_cols y X_cols duplicados t_cols se usa en dataloader X_cols para indexar con f_cols
#.      idea mantenemos solo X_cols y en el dataloader corregimos con 'y' y 'insample_mask' 
# TODO: assert no nans in numpy tensors en get_filter_tensor
class TimeSeriesDataset(Dataset):
    def __init__(self, #TODO: poner hint the tipo
                 y_df,
                 X_df = None,
                 S_df = None,
                 f_cols = None,
                 ts_train_mask = None):
        """
        """
        assert type(y_df) == pd.core.frame.DataFrame
        assert all([(col in y_df) for col in ['unique_id', 'ds', 'y']])
        if X_df is not None:
            assert type(X_df) == pd.core.frame.DataFrame
            assert all([(col in X_df) for col in ['unique_id', 'ds']])

        print('Processing dataframes ...')
        # Pandas dataframes to lists
        ts_data, x_s, self.meta_data, self.t_cols, self.X_cols = self._df_to_lists(y_df=y_df, S_df=S_df, X_df=X_df)

        # Attributes
        self.n_series = len(ts_data)
        self.max_len = max([len(ts['y']) for ts in ts_data])
        self.n_channels = len(ts_data[0].values())
        self.frequency = pd.infer_freq(y_df.head()['ds']) #TODO: improve, can die with head
        self.f_cols = f_cols

        # Number of X and S features
        self.n_x, self.n_s = 0, 0
        if X_df is not None:
            self.n_x = len(self.X_cols)
        if S_df is not None:
            self.n_s = S_df.shape[1]-1 # 1 for unique_id

        print('Creating ts tensor ...')
        self.ts_tensor, self.x_s, self.len_series = self._create_tensor(ts_data, x_s)

        if ts_train_mask is not None:
            assert len(ts_train_mask)==self.max_len, f'Outsample mask must have length {self.max_len}'
        else:
            ts_train_mask = np.ones(self.max_len)

        self._declare_outsample_train_mask(ts_train_mask)


    def _df_to_lists(self, y_df, S_df, X_df):
        """
        """
        unique_ids = y_df['unique_id'].unique()

        if X_df is not None:
            X_cols = [col for col in X_df.columns if col not in ['unique_id','ds']]
        else:
            X_cols = []

        if S_df is not None:
            S_cols = [col for col in S_df.columns if col not in ['unique_id']]
        else:
            S_cols = []

        ts_data = []
        x_s = []
        meta_data = []
        for i, u_id in enumerate(unique_ids):
            top_row = np.asscalar(y_df['unique_id'].searchsorted(u_id, 'left'))
            bottom_row = np.asscalar(y_df['unique_id'].searchsorted(u_id, 'right'))
            serie = y_df[top_row:bottom_row]['y'].values
            last_ds_i = y_df[top_row:bottom_row]['ds'].max()
            
            # Y values
            ts_data_i = {'y': serie}
            
            # X values
            for X_col in X_cols:
                serie =  X_df[top_row:bottom_row][X_col].values
                ts_data_i[X_col] = serie
            ts_data.append(ts_data_i)

            # S values
            s_data_i = defaultdict(list)
            for S_col in S_cols:
                s_data_i[S_col] = S_df.loc[S_df['unique_id']==u_id, S_col].values
            x_s.append(s_data_i)

            # Metadata
            meta_data_i = {'unique_id': u_id,
                           'last_ds': last_ds_i}
            meta_data.append(meta_data_i)

        t_cols = ['y'] + X_cols + ['insample_mask', 'outsample_mask']

        return ts_data, x_s, meta_data, t_cols, X_cols

    def _create_tensor(self, ts_data, x_s):
        """
        ts_tensor: n_series x n_channels x max_len
        """
        ts_tensor = np.zeros((self.n_series, self.n_channels + 2, self.max_len)) # +2 for the masks
        static_tensor = np.zeros((self.n_series, len(x_s[0])))

        len_series = []
        for idx in range(self.n_series):
            ts_idx = np.array(list(ts_data[idx].values()))
            ts_tensor[idx, :self.t_cols.index('insample_mask'), -ts_idx.shape[1]:] = ts_idx
            ts_tensor[idx, self.t_cols.index('insample_mask'), -ts_idx.shape[1]:] = 1
            # To avoid sampling windows without inputs to predict
            # Outsample mask will be later completed with the 'train_mask'
            ts_tensor[idx, self.t_cols.index('outsample_mask'), -(ts_idx.shape[1]-1):] = 1
            static_tensor[idx, :] = list(x_s[idx].values())
            len_series.append(ts_idx.shape[1])

        return ts_tensor, static_tensor, np.array(len_series)

    def _declare_outsample_train_mask(self, ts_train_mask):
        # Update attribute and ts_tensor
        self.ts_train_mask = ts_train_mask

    def get_meta_data_col(self, col):
        """
        """
        col_values = [x[col] for x in self.meta_data]
        return col_values

def get_filtered_tensor(self, offset, output_size, window_sampling_limit, ts_idxs=None):
        """
        Esto te da todo lo que tenga el tensor, el futuro incluido esto orque se usa exogenoas del futuro
        La mascara se hace despues
        """
        last_outsample_ds = self.max_len - offset + output_size
        first_ds = max(last_outsample_ds - window_sampling_limit - output_size, 0)
        if ts_idxs is None:
            filtered_tensor = self.ts_tensor[:, :, first_ds:last_outsample_ds]
        else:
            filtered_tensor = self.ts_tensor[ts_idxs, :, first_ds:last_outsample_ds]
        right_padding = max(last_outsample_ds - self.max_len, 0) #To padd with zeros if there is "nothing" to the right

        train_mask = self.ts_train_mask[first_ds:last_outsample_ds]

        return filtered_tensor, right_padding, train_mask

def get_f_idxs(self, cols):
        # Check if cols are available future variables and return idxs
        assert all(col in self.f_cols for col in cols), f'Some variables in {cols} are not available in f_cols.'
        f_idxs = [self.X_cols.index(col) for col in cols]
        return f_idxs


In [4]:
from nixtla.data.datasets.tourism import Tourism, TourismInfo
tourism_dataset = Tourism.load(directory='data', group=TourismInfo.groups[0])
tourism_dataset.ts_tensor

Processing dataframes ...
Creating ts tensor ...


array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         3.73856371e+04, 3.84319699e+04, 4.03453300e+04],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         3.91049000e+05, 4.54041000e+05, 5.52942000e+05],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.80507000e+03, 2.13865000e+03, 2.68214000e+03],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [0.00000000e+00, 0.00000000e

In [5]:
tourism_dataset.t_cols

['y', 'insample_mask', 'outsample_mask']

In [6]:
tourism_dataset.ts_train_mask

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [7]:
tourism_dataset.ts_tensor[0,-1,:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [8]:
tourism_dataset.ts_tensor[0,0,:]

array([    0.    ,     0.    ,     0.    ,     0.    ,     0.    ,
           0.    ,     0.    ,     0.    ,     0.    ,     0.    ,
           0.    ,     0.    ,     0.    ,     0.    ,     0.    ,
           0.    ,     0.    ,     0.    ,     0.    ,     0.    ,
           0.    ,     0.    ,     0.    ,     0.    ,     0.    ,
           0.    ,     0.    ,     0.    ,     0.    ,     0.    ,
           0.    ,     0.    , 25092.2284, 24271.5134, 25828.9883,
       27697.5047, 27956.2276, 29924.4321, 30216.8321, 32613.4968,
       36053.1674, 38472.7532, 38420.894 , 36555.6156, 37385.6371,
       38431.9699, 40345.33  ])

In [9]:
tourism_dataset.ts_train_mask[:]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
# def rolling_sum(a, window_size=4) :
#     ret = np.cumsum(np.concatenate((M, np.zeros(shape=(len(M), window_size-1))), axis=1), 
#                     axis=1, dtype=float)
#     ret[:, window_size:] = ret[:, window_size:] - ret[:, :-window_size]
#     return ret[:, window_size-1:]

# M = np.array([[0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.],
#               [0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
#               [1.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.]])

# print(rolling_sum(M))